In [ ]:
!pip install ../input/torchlibrosa/torchlibrosa-0.0.5-py3-none-any.whl > /dev/null

In [ ]:
!pip install colorednoise > /dev/null

In [ ]:
import os
import ast
import numpy as np
import warnings
from sklearn import metrics
from torch.utils.data import Dataset
import pandas as pd
import torchaudio
import glob
import colorednoise as cn
from sklearn.model_selection import StratifiedKFold, GroupKFold
import os
from torch.utils.data import Dataset
import pandas as pd
import torchaudio
from sklearn.preprocessing import MultiLabelBinarizer
from fastai.vision.all import *
from typing import Optional,Tuple,List
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
from timm import create_model
import audioread
import librosa
import soundfile as sf
from albumentations.core.transforms_interface import ImageOnlyTransform
from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation

import albumentations as A
import albumentations.pytorch.transforms as T
import matplotlib.pyplot as plt

In [ ]:
class AllParams:
    seed = 71
    target_columns = 'afrsil1 akekee akepa1 akiapo akikik amewig aniani apapan arcter \
                      barpet bcnher belkin1 bkbplo bknsti bkwpet blkfra blknod bongul \
                      brant brnboo brnnod brnowl brtcur bubsan buffle bulpet burpar buwtea \
                      cacgoo1 calqua cangoo canvas caster1 categr chbsan chemun chukar cintea \
                      comgal1 commyn compea comsan comwax coopet crehon dunlin elepai ercfra eurwig \
                      fragul gadwal gamqua glwgul gnwtea golphe grbher3 grefri gresca gryfra gwfgoo \
                      hawama hawcoo hawcre hawgoo hawhaw hawpet1 hoomer houfin houspa hudgod iiwi incter1 \
                      jabwar japqua kalphe kauama laugul layalb lcspet leasan leater1 lessca lesyel lobdow lotjae \
                      madpet magpet1 mallar3 masboo mauala maupar merlin mitpar moudov norcar norhar2 normoc norpin \
                      norsho nutman oahama omao osprey pagplo palila parjae pecsan peflov perfal pibgre pomjae puaioh \
                      reccar redava redjun redpha1 refboo rempar rettro ribgul rinduc rinphe rocpig rorpar rudtur ruff \
                      saffin sander semplo sheowl shtsan skylar snogoo sooshe sooter1 sopsku1 sora spodov sposan \
                      towsol wantat1 warwhe1 wesmea wessan wetshe whfibi whiter whttro wiltur yebcar yefcan zebdov'.split()

    img_size = 224 # 128
    main_metric = "epoch_f1_at_03"
    USE_SEC = 30
    # Melspectrogram
    period = 5
    duration = period
    n_mels = 224 # 128
    fmin = 20
    fmax = 16000
    n_fft = 2048
    hop_length = 512
    sample_rate = 32000
    sr = sample_rate
    melspectrogram_parameters = {
        "n_mels": 224, # 128,
        "fmin": 20,
        "fmax": 16000
    }
    
    # train
    epochs = 9
    folds = [0] # [0, 1, 2, 3, 4]
    N_FOLDS = 5
    LR = 1e-3
    ETA_MIN = 1e-6
    WEIGHT_DECAY = 0.0001
    batch_size = 32 # 32
    base_model_name = "tf_efficientnet_b0_ns"
    EARLY_STOPPING = True
    DEBUG = False # True
    EVALUATION = 'AUC'
    apex = True
    pooling = "max"
    pretrained = True
    num_classes = 152
    in_channels = 3
    mode = 'train'
    mixup_or_cutmix_epochs = 6
    mixup_or_cutmix = 'mixup'
    current_epoch = 0
    
params = AllParams

In [ ]:
import os

OUTPUT_DIR = f'./'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
   
    
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(params.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# loading training data

In [ ]:
import glob

all_path = glob.glob('../input/birdclef-2022/train_audio/*/*.ogg')

len(all_path)

In [ ]:
import ast


train = pd.read_csv('../input/birdclef-2022/train_metadata.csv')


train['new_target'] = train['primary_label'] + ' ' + train['secondary_labels'].map(lambda x: ' '.join(ast.literal_eval(x)))
train['len_new_target'] = train['new_target'].map(lambda x: len(x.split()))
# train['len_new_target'].value_counts()
train.head()

In [ ]:
path_df = pd.DataFrame(all_path, columns=['file_path'])
path_df['filename'] = path_df['file_path'].map(lambda x: x.split('/')[-2]+'/'+x.split('/')[-1])
path_df.head()

In [ ]:
train = pd.merge(train, path_df, on='filename')
print(train.shape)
train.head()

In [ ]:
#train = train[train.primary_label.isin(scored_classes)]
#train.reset_index(inplace=True)

In [ ]:
Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (trn_index, val_index) in enumerate(Fold.split(train, train['primary_label'])):
    train.loc[val_index, 'kfold'] = int(n)
train['kfold'] = train['kfold'].astype(int)

In [ ]:
train.to_csv('train_folds.csv', index=False)

# Creating dataset

In [ ]:
class NoiseInjection(Transform):
    def __init__(self, always_apply=False, p=0.5, max_noise_level=0.5):
        self.noise_level = (0.0, max_noise_level)
        self.p = p
        self.always_apply = always_apply
        
    def encodes(self, signal: np.ndarray):
        noise_level = np.random.uniform(*self.noise_level)
        noise = np.random.randn(len(signal))
        if self.always_apply == True:
                signal = (signal + noise * noise_level).astype(signal.dtype)
            
        else:
            if np.random.rand() < self.p:
                signal = (signal + noise * noise_level).astype(signal.dtype) 
        return signal


class GaussianNoise(Transform):
    def __init__(self, always_apply=False, p=0.5, min_snr=5, max_snr=20):
        self.always_apply = always_apply
        self.p = p
        self.min_snr = min_snr
        self.max_snr = max_snr

    def encodes(self, signal: np.ndarray):
        snr = np.random.uniform(self.min_snr, self.max_snr)
        a_signal = np.sqrt(signal ** 2).max()
        a_noise = a_signal / (10 ** (snr / 20))
        white_noise = np.random.randn(len(signal))
        a_white = np.sqrt(white_noise ** 2).max()
        
        if self.always_apply == True:
            signal = (signal + white_noise * 1 / a_white * a_noise).astype(signal.dtype)
            
        else:
            if np.random.rand() < self.p:
                signal = (signal + white_noise * 1 / a_white * a_noise).astype(signal.dtype)
        return signal


class PinkNoise(Transform):
    def __init__(self, always_apply=False, p=0.5, min_snr=5, max_snr=20):
        self.always_apply = always_apply
        self.p = p
        self.min_snr = min_snr
        self.max_snr = max_snr

    def encodes(self, signal: np.ndarray):
        snr = np.random.uniform(self.min_snr, self.max_snr)
        a_signal = np.sqrt(signal ** 2).max()
        a_noise = a_signal / (10 ** (snr / 20))

        pink_noise = cn.powerlaw_psd_gaussian(1, len(signal))
        a_pink = np.sqrt(pink_noise ** 2).max()
        if self.always_apply == True:
            signal = (signal + pink_noise * 1 / a_pink * a_noise).astype(signal.dtype)
            
        else:
            if np.random.rand() < self.p:
                signal = (signal + pink_noise * 1 / a_pink * a_noise).astype(signal.dtype)
        return signal

class RandomVolume(Transform):
    def __init__(self,always_apply=False, p=0.2, limit=12):
        self.limit = limit
        self.p = p
        self.always_apply = always_apply
    def encodes(self, signal: np.ndarray):
        db = np.random.uniform(-self.limit, self.limit)
        if self.always_apply == True:
            if db >= 0:
                return self.volume_up(signal, db)
            else:
                return self.volume_down(signal, db)
        else:
            if np.random.rand() < self.p:
                if db >= 0:
                    return self.volume_up(signal, db)
                else:
                    return self.volume_down(signal, db)      
            return signal

    def volume_down(self,signal: np.ndarray, db: float):
        signal = signal * self._db2float(-db)
        return signal
    
    def volume_up(self,signal: np.ndarray, db: float):
        signal = signal * self._db2float(db)
        return signal
    
    def _db2float(self,db: float, amplitude=True):
        if amplitude:
            return 10 ** (db / 20)
        else:
            return 10 ** (db / 10)

class TimeShift(Transform):
    def __init__(self, always_apply=False, p=0.5, max_shift_second=2,padding_mode="zero"):
        self.p = p
        self.max_shift_second = max_shift_second
        self.padding_mode = padding_mode
        self.always_apply = always_apply
        self.sr = params.sr
        assert padding_mode in ["replace", "zero"], "`padding_mode` must be either 'replace' or 'zero'"
        
    def encodes(self, signal: np.ndarray):
        shift = np.random.randint(-self.sr * self.max_shift_second, self.sr * self.max_shift_second)
        augmented = np.roll(signal, shift)
        if self.always_apply == True:
            if self.padding_mode == "zero":
                if shift > 0:
                    augmented[:shift] = 0
                else:
                    augmented[shift:] = 0
            return augmented
        else:
            if np.random.rand() < self.p:
                if self.padding_mode == "zero":
                    if shift > 0:
                        augmented[:shift] = 0
                    else:
                        augmented[shift:] = 0
                    return augmented
        return signal   

class PitchShift(Transform):
    def __init__(self, always_apply=False, p=0.5, max_steps=5):
        self.p = p
        self.always_apply = always_apply
        self.max_steps = max_steps
        self.sr = params.sr

    def encodes(self, signal: np.ndarray):
        n_steps = np.random.randint(-self.max_steps, self.max_steps)
        if self.always_apply == True:
            augmented = librosa.effects.pitch_shift(signal, sr=self.sr, n_steps=n_steps)
            return augmented
        elif np.random.rand() < self.p:
            augmented = librosa.effects.pitch_shift(signal, sr=self.sr, n_steps=n_steps)
            return augmented          
        else:
            return signal

In [ ]:

mean = (0.485, 0.456, 0.406) # RGB
std = (0.229, 0.224, 0.225) # RGB

albu_transforms = {
    'train' : A.Compose([
            A.HorizontalFlip(p=0.5),
            A.OneOf([
                A.Cutout(max_h_size=5, max_w_size=16),
                A.CoarseDropout(max_holes=4),
            ], p=0.5),
            A.Normalize(mean, std),
    ]),
    'valid' : A.Compose([
            A.Normalize(mean, std),
    ]),
}

In [ ]:
class OneOf(Transform):
    def __init__(self, transforms,p=0.5):
        self.p = p
        self.transforms = transforms
        
        transforms_ps = [t.p for t in transforms]
        s = sum(transforms_ps)
        self.transforms_ps = [t / s for t in transforms_ps]
        
    def encodes(self, signal: np.ndarray):
        if params.mode == "train":
            if self.transforms_ps and (random.random() < self.p):
                random_state = np.random.RandomState(random.randint(0, 2 ** 32 - 1))
                t = random_state.choice(self.transforms, p=self.transforms_ps)
                signal = t(signal)

            signal = RandomVolume(p=0.2, limit=12)(signal)
        signal =  self.Normalize(signal=signal,p=1)
        signal = np.concatenate([signal, signal, signal])[:params.duration * params.sr] 

        if params.mode == "train":
            train = True
        else:
            train = False
        signal = self.crop_or_pad(signal, params.duration * params.sr, sr=params.sr, train=train, probs=None)
        return signal
    
    def crop_or_pad(self,signal, length, sr, train, probs=None):
        if len(signal) <= length:
            signal = np.concatenate([signal, np.zeros(length - len(signal))])
        else:
            if train == False:
                start = 0
            elif probs is None:
                start = np.random.randint(len(signal) - length)
            else:
                start = (
                        np.random.choice(np.arange(len(probs)), p=probs) + np.random.random()
                )
                start = int(sr * (start))

            signal = signal[start: start + length]

        return signal.astype(np.float32) 
    
    def Normalize(self,signal,always_apply=False, p=1):
        max_vol = np.abs(signal).max()
        if always_apply == True:
             signal_vol = signal * 1 / max_vol

        else:
            if np.random.rand() < p:
                 signal_vol = signal * 1 / max_vol 
        return np.array(signal_vol)
    
    
    
class get_audio_sample_path(Transform):
    def encodes(self, x):
        return x.file_path
    
class get_audio_sample_label(Transform):       
    def encodes(self,x):
        labels = x.new_target
        targets = np.zeros(len(params.target_columns), dtype=float)
        for ebird_code in labels.split():
            targets[params.target_columns.index(ebird_code)] = 1.0
        return targets

class load_signal(Transform):
    def __init__(self,device='cpu'):self.device=device
    def encodes(self, x:str):
        sr = params.sample_rate
        signal, sr = sf.read(x, always_2d=True)
        #signal = np.load(x)
        signal = np.mean(signal, 1) # there is (X, 2) array
        if len(signal) > sr * params.USE_SEC:
            signal = signal[sr:-sr]
            signal = signal[:sr * params.USE_SEC]

        #if len(signal) > 0:
        #    signal = signal[:params.duration*sr]
        #else:
        #    signal = np.array([0.0001 for x in range( params.USE_SEC*sr)])
        return signal


class mel_spec(Transform):
    def encodes(self, x):
        melspec = librosa.feature.melspectrogram(
            y=x, sr=params.sample_rate, n_mels=params.n_mels, fmin=params.fmin, fmax=params.fmax,
        )
        image = librosa.power_to_db(melspec).astype(np.float32)
        image = self.mono_to_color(image)
        image = image.astype(np.uint8)
        # image = np.load(wav_path) # (224, 313, 3)
        image = albu_transforms[params.mode](image=image)['image']
        image = image.T
        return image

    def mono_to_color(self,X, eps=1e-6, mean=None, std=None):
        X = np.stack([X, X, X], axis=-1)

        # Standardize
        mean = mean or X.mean()
        std = std or X.std()
        X = (X - mean) / (std + eps)

        # Normalize to [0, 255]
        _min, _max = X.min(), X.max()

        if (_max - _min) > eps:
            V = np.clip(X, _min, _max)
            V = 255 * (V - _min) / (_max - _min)
            V = V.astype(np.uint8)
        else:
            V = np.zeros_like(X, dtype=np.uint8)
        return V

In [ ]:
if torch.cuda.is_available():
        device = "cuda"
else:
        device = "cpu"
print(f"Using {device}")

In [ ]:
val_fold = 0
def get_dls(val_fold=0,bs=64):
    splits = [train[train.kfold != val_fold].index.tolist() , train[train.kfold == val_fold].index.tolist()]
    #x_tfms = [get_audio_sample_path,load_signal(device),resample_if_necessary(32000),
    #          mix_down_if_necessary,cut_if_necessary(),right_pad_if_necessary,
    #          mel_spec,mono_to_color]
    x_tfms = [get_audio_sample_path,load_signal,OneOf([   
                            NoiseInjection(p=1, max_noise_level=0.04),
                            #GaussianNoise(p=1, min_snr=5, max_snr=20),
                            #PinkNoise(p=1, min_snr=5, max_snr=20),
                            PitchShift(max_steps=4, p=0.1),
                            TimeShift(max_shift_second=0.5, p=0.1)
                        ],p=1.),mel_spec]
    
    y_tfms = [get_audio_sample_label]

    dsets = Datasets(items = train ,tfms=[x_tfms, y_tfms],splits=splits)

    dls = dsets.dataloaders(bs=bs)   
    return dls

In [ ]:
xb, yb = get_dls(val_fold=0,bs=params.batch_size).one_batch()
xb.shape, yb.shape

In [ ]:
dls = get_dls(val_fold=0,bs=params.batch_size)
dls.vocab = params.target_columns

# Creating the model

In [ ]:
def interpolate(x: torch.Tensor, ratio: int):
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    output = F.interpolate(
        framewise_output.unsqueeze(1),
        size=(frames_num, framewise_output.size(2)),
        align_corners=True,
        mode="bilinear").squeeze(1)

    return output

In [ ]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)

class AttBlockV2(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [ ]:
import random
class building_model(Module):
    def __init__(self,num_classes:int,arch:str = params.base_model_name, pretrained:bool=False):
        self.spec_augmenter = SpecAugmentation(time_drop_width=64//2, time_stripes_num=2,
                                                   freq_drop_width=8//2, freq_stripes_num=2)
        self.spec_time_augmenter = torchaudio.transforms.TimeMasking(24)
        self.spec_freq_augmenter = torchaudio.transforms.FrequencyMasking(96)
        self.bn0 = nn.BatchNorm2d(params.n_mels)
        self.base_model = create_model(arch, pretrained=pretrained)
        self.linear = nn.Linear(self.base_model.get_classifier().out_features, num_classes)
        self.layers = list(self.base_model.children())[:-2]
        self.encoder = nn.Sequential(*self.layers)
        self.in_features = self.base_model.classifier.in_features
        self.fc1 = nn.Linear(self.in_features, self.in_features, bias=True)
        self.fc2 = nn.Linear(num_classes,num_classes, bias=False)
        self.att_block = AttBlockV2(self.in_features, num_classes, activation="sigmoid")  
        self.init_weight()
        self.mode = 'train'
    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        
    def change_mode(self,mode='train'):
        self.mode = mode

    def forward(self,input_data):
        frames_num = input_data.shape[2]
        x = input_data # (batch_size, 3, time_steps, mel_bins)
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            if random.random() < 0.75:
                x = self.spec_augmenter(x)
                #x = self.spec_time_augmenter(x)
                #x = self.spec_freq_augmenter(x)
                
        x = x.transpose(2, 3)
        x = self.encoder(x)  
        x = torch.mean(x, dim=3)
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        x = F.dropout(x, p=0.4, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.4, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        interpolate_ratio = frames_num // segmentwise_output.size(1)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            'framewise_output': framewise_output,
            'clipwise_output': clipwise_output,
            'logit': logit,
            'framewise_logit': framewise_logit,
        }

        return output_dict    

# Loss

In [ ]:
# https://www.kaggle.com/c/rfcx-species-audio-detection/discussion/213075
class BCEFocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, preds, targets):
        bce_loss = nn.BCEWithLogitsLoss(reduction='none')(preds, targets)
        probas = torch.sigmoid(preds)
        loss = targets * self.alpha * \
            (1. - probas)**self.gamma * bce_loss + \
            (1. - targets) * probas**self.gamma * bce_loss
        loss = loss.mean()
        return loss


class BCEFocal2WayLoss(nn.Module):
    def __init__(self, weights=[1, 1], class_weights=None):
        super().__init__()

        self.focal = BCEFocalLoss()

        self.weights = weights

    def forward(self, input, target):
        input_ = input["logit"]
        target = target.float()

        framewise_output = input["framewise_logit"]
        clipwise_output_with_max, _ = framewise_output.max(dim=1)

        loss = self.focal(input_, target)
        aux_loss = self.focal(clipwise_output_with_max, target)

        return self.weights[0] * loss + self.weights[1] * aux_loss

In [ ]:
def loss_fn(logits, targets):
    if params.mode == "valid":
        loss_fct = BCEFocal2WayLoss()
        loss = loss_fct(logits, targets)
        return loss
    elif (params.mixup_or_cutmix == "mixup" and params.current_epoch < params.mixup_or_cutmix_epochs):
        loss = mixup_criterion(logits, targets)
        return loss
    elif (params.mixup_or_cutmix == "cutmix" and params.current_epoch < params.mixup_or_cutmix_epochs):
        loss = cutmix_criterion(logits, targets)
        return loss
    else:
        loss_fct = BCEFocal2WayLoss()
        loss = loss_fct(logits, targets)
        return loss        
        
def cutmix_criterion(preds, new_targets):
    targets1, targets2, lam = new_targets[0], new_targets[1], new_targets[2]
    criterion = BCEFocal2WayLoss()
    return lam * criterion(preds, targets1) + (1 - lam) * criterion(preds, targets2)

def mixup_criterion(preds, new_targets):
    targets1, targets2, lam = new_targets[0], new_targets[1], new_targets[2]
    criterion = BCEFocal2WayLoss()
    return lam * criterion(preds, targets1) + (1 - lam) * criterion(preds, targets2)


# metric

In [ ]:
def MetricMeter(y_pred, y_true):
    with torch.no_grad():
        y_true = y_true.cpu().detach().numpy()
        y_pred = y_pred["clipwise_output"].cpu().detach().numpy()
        f1_03 = metrics.f1_score(np.array(y_true),np.array(y_pred) > 0.3, average="micro")        
        return f1_03

In [ ]:
import numpy as np
import sklearn.metrics

def comp_metric(y_pred, y_true, epsilon=1e-9):
    with torch.no_grad():
        y_true = y_true.cpu().numpy()
        y_pred = y_pred["clipwise_output"].cpu().numpy()
    # Get representative confusion matrices for each label
    mlbl_cms = sklearn.metrics.multilabel_confusion_matrix(y_true, np.array(y_pred) > 0.3)

    # Get two scores (TP and TN SCORES)
    tp_scores = np.array([
        mlbl_cm[1, 1]/(epsilon+mlbl_cm[:, 1].sum()) \
        for mlbl_cm in mlbl_cms
        ])
    tn_scores = np.array([
        mlbl_cm[0, 0]/(epsilon+mlbl_cm[:, 0].sum()) \
        for mlbl_cm in mlbl_cms
        ])

    # Get average
    tp_mean = tp_scores.mean()
    tn_mean = tn_scores.mean()

    return round((tp_mean+tn_mean)/2, 8)


In [ ]:
class train_val_clb(Callback):
        def after_epoch(self):
            params.current_epoch = params.current_epoch + 1
            print(params.current_epoch)
            
        def after_fit(self):
            params.current_epoch = 0
            
        def after_validate(self):
            self.learn.model.change_mode('train')
            params.mode = 'train'
            
        def before_validate(self):
            self.learn.model.change_mode('valid')
            params.mode = 'valid'
            
        def before_batch(self):
            if params.mode == "train":
                self.learn.dls.bs = 32
                data = self.learn.xb
                targets = self.learn.yb
                if params.current_epoch < params.mixup_or_cutmix_epochs:
                    if np.random.rand()<0.5:
                        new_data, new_targets = self.mixup(data[0],targets[0],0.4)
                        self.learn.xb = (new_data,)
                        self.learn.yb = (new_targets,)
                        params.mixup_or_cutmix = "mixup"

                    else:
                        new_data, new_targets = self.cutmix(data[0], targets[0],0.4)
                        self.learn.xb = (new_data,)
                        self.learn.yb = (new_targets,)
                        params.mixup_or_cutmix = "cutmix"
 
            else:
                self.learn.dls.bs = 32

        def cutmix(self,data, targets, alpha):
            indices = torch.randperm(data.size(0))
            shuffled_data = data[indices]
            shuffled_targets = targets[indices]

            lam = np.random.beta(alpha, alpha)
            bbx1, bby1, bbx2, bby2 = self.rand_bbox(data.size(), lam)
            data[:, :, bbx1:bbx2, bby1:bby2] = data[indices, :, bbx1:bbx2, bby1:bby2]
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))

            new_targets = [targets, shuffled_targets, lam]
            return data, new_targets

        def mixup(self,data, targets, alpha):
            indices = torch.randperm(data.size(0))
            shuffled_data = data[indices]
            shuffled_targets = targets[indices]

            lam = np.random.beta(alpha, alpha)
            new_data = data * lam + shuffled_data * (1 - lam)
            new_targets = [targets, shuffled_targets, lam]
            return new_data, new_targets
        
        def rand_bbox(self,size, lam):
            W = size[2]
            H = size[3]
            cut_rat = np.sqrt(1. - lam)
            cut_w = int(W * cut_rat)
            cut_h = int(H * cut_rat)

            # uniform
            cx = np.random.randint(W)
            cy = np.random.randint(H)

            bbx1 = np.clip(cx - cut_w // 2, 0, W)
            bby1 = np.clip(cy - cut_h // 2, 0, H)
            bbx2 = np.clip(cx + cut_w // 2, 0, W)
            bby2 = np.clip(cy + cut_h // 2, 0, H)

            return bbx1, bby1, bbx2, bby2

In [ ]:
model = building_model(num_classes=len(dls.vocab),pretrained=params.pretrained)
#model = resnet18(num_classes=152)
learn = Learner(dls,model,loss_func=loss_fn,metrics=MetricMeter,cbs=[train_val_clb,SaveModelCallback(monitor='MetricMeter',comp=np.greater)])
#learn.summary()

In [ ]:
#learn.lr_find()

In [ ]:
learn.model = load_learner("../input/fastai-effb0-base-model/model.pkl").model

In [ ]:
learn.fit_one_cycle(10,lr_max = params.LR,wd=params.WEIGHT_DECAY)

In [ ]:
learn.export(fname='model.pkl')

# references

1- https://www.kaggle.com/code/kaerunantoka/birdclef2022-audio-to-numpy-1-4/notebook

2- https://www.kaggle.com/code/kaerunantoka/birdclef2022-n001-training/notebook